# Bsky integration

In [13]:
//| export

import { AtpAgent } from "npm:@atproto/api";
import type { AppBskyFeedDefs, AppBskyFeedPost } from "npm:@atproto/api";

In [4]:
import "jsr:@std/dotenv/load";

[Module: null prototype] {  }

In [38]:
const agent = new AtpAgent({ service: "https://bsky.social" });
await agent.login({
  identifier: Deno.env.get("BSKY_AGENT_IDENTIFIER"),
  password: Deno.env.get("BSKY_AGENT_PASSWORD"),
});

XRPCResponse {
  data: {
    did: "did:plc:mqy3jltibqanjnqx7cxbrdpo",
    didDoc: {
      "@context": [
        "https://www.w3.org/ns/did/v1",
        "https://w3id.org/security/multikey/v1",
        "https://w3id.org/security/suites/secp256k1-2019/v1"
      ],
      id: "did:plc:mqy3jltibqanjnqx7cxbrdpo",
      alsoKnownAs: [ "at://c2img.bsky.social" ],
      verificationMethod: [
        {
          id: "did:plc:mqy3jltibqanjnqx7cxbrdpo#atproto",
          type: "Multikey",
          controller: "did:plc:mqy3jltibqanjnqx7cxbrdpo",
          publicKeyMultibase: "zQ3shTGDSg1fkeXqqp82kkH19N2LpB7pAJkHJxKSvahTztSwU"
        }
      ],
      service: [
        {
          id: "#atproto_pds",
          type: "AtprotoPersonalDataServer",
          serviceEndpoint: "https://brittlegill.us-west.host.bsky.network"
        }
      ]
    },
    handle: "c2img.bsky.social",
    email: "philip.nuzhnyy+c2i@gmail.com",
    emailConfirmed: true,
    emailAuthFactor: false,
    accessJwt: "eyJ0eXAiOiJ

# Post URL to @at protocol URI

Figure out how to convert bsky.app URL to a URI we can use to fetch data. We
need to go from this:

```
https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d
```

To smth like this

```
at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
```

`did:plc:ubdeopbbkbgedccgbum7dhsh` is DID for callmephilip.com handle, which
needs to be resolved first

In [6]:
//| export

const postURLToAtpURI = async (
  postUrl: string,
  agent: AtpAgent,
): Promise<string> => {
  const urlParts = new URL(postUrl);
  const pathParts = urlParts.pathname.split("/");
  const h = await agent.resolveHandle({ handle: pathParts[2] });
  return `at://${h.data.did}/app.bsky.feed.post/${pathParts[4]}`;
};

In [7]:
await postURLToAtpURI(
  "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
  agent,
);

"at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"

# Grabbing thread data

Thread has a bunch of nested posts inside replies. Unwrap this into a list of
posts

In [8]:
//| export

const unwrapThreadPosts = (
  thread: AppBskyFeedDefs.ThreadViewPost,
): AppBskyFeedPost[] => {
  const posts: AppBskyFeedPost[] = [];

  // Add root post
  if (thread.post) {
    posts.push(thread.post);
  }

  // Recursively handle replies
  if (thread.replies) {
    thread.replies.forEach((reply) => {
      posts.push(...unwrapThreadPosts(reply));
    });
  }

  // Handle nested reply if present
  if ("parent" in thread && thread.parent) {
    posts.push(...unwrapThreadPosts(thread.parent));
  }

  return posts;
};

export const downloadThread = async (
  postUrl: string,
  agent: AtpAgent,
): AppBskyFeedPost[] => {
  const d = await agent.getPostThread({
    uri: await postURLToAtpURI(postUrl, agent),
  });
  return unwrapThreadPosts(d.data.thread);
};

In [9]:
const posts = await downloadThread(
  "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
  agent,
);

# Converting posts to MD

The most basic setup is when there is just a piece of text display. More complex
cases will have links, attachments

In [ ]:
//| export

export const postToMd = (post: AppBskyFeedPost): string => {
  const text = post.record.text;
  return `
    # ${post.author.displayName} (@${post.author.handle}) - ${post.record.createdAt}

    ${text}
  `;
};

In [27]:
await Deno.jupyter.display(
  {
    "text/markdown": postToMd(posts[0]),
  },
  { raw: true },
);


    # Philip Nuzhnyi (@callmephilip.com)

    🧵 Notes on Deno + Jupyter 🦕 🔭
  

In [37]:
await Deno.jupyter.display(
  {
    "text/markdown": posts.reduce((acc, post) => {
      return acc + postToMd(post);
    }, ""),
  },
  { raw: true },
);


    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:46:20.768Z

    🧵 Notes on Deno + Jupyter 🦕 🔭
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:49:32.248Z

    Jupyter kernel for Deno - Deno docs docs.deno.com/runtime/refe...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:56:32.679Z

    Repro of original Deno2 jupyter demo with broken mermaid integration - gist.github.com/aaronblondea...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:00:14.858Z

    Rich displays for jupyter js kernels - deno.land/x/display@v1...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:26:17.631Z

    @kylekelley.bsky.social has a bunch of Deno in notebook examples here - github.com/rgbkrk/denot...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:44:33.927Z

    nice video showing deno + jupyter (via noteable RIP) www.youtube.com/watch?v=b5OK...
  
    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:53:55.726Z

    anywidget for deno jupyter kernel - jsr.io/@anywidget/d...
  

In [39]:
//| export

export const downloadPostToMd = async (postUrl: string): Promise<string> => {
  const agent = new AtpAgent({ service: "https://bsky.social" });

  await agent.login({
    identifier: Deno.env.get("BSKY_AGENT_IDENTIFIER"),
    password: Deno.env.get("BSKY_AGENT_PASSWORD"),
  });

  const posts = await downloadThread(
    postUrl,
    agent,
  );

  return posts.reduce((acc, post) => {
    return acc + postToMd(post);
  }, "");
};

In [40]:
await downloadPostToMd(
  "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
);

"\n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:46:20.768Z\n" +
  "\n" +
  "    🧵 Notes on Deno + Jupyter 🦕 🔭\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:49:32.248Z\n" +
  "\n" +
  "    Jupyter kernel for Deno - Deno docs docs.deno.com/runtime/refe...\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:56:32.679Z\n" +
  "\n" +
  "    Repro of original Deno2 jupyter demo with broken mermaid integration - gist.github.com/aaronblondea...\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:00:14.858Z\n" +
  "\n" +
  "    Rich displays for jupyter js kernels - deno.land/x/display@v1...\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:26:17.631Z\n" +
  "\n" +
  "    @kylekelley.bsky.social has a bunch of Deno in notebook examples here - github.com/rgbkrk/denot...\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:44:33.927Z\n" +
  "\n" +
  "  